In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2)

# Load saved model state
model.load_state_dict(torch.load('models/best_model_state.bin'))
model.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Mocca\AppData\Local\Temp\ipykernel_17776\1500747696.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serial

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [3]:
def prepare_input_consistent(doc, summ, tokenizer, max_length=512):
    # Tokenize the document and summary
    doc_tokens = tokenizer.tokenize(doc)
    summ_tokens = tokenizer.tokenize(summ)

    # Ensure total length fits within max_length
    combined_tokens = (
        [tokenizer.cls_token_id] +
        tokenizer.convert_tokens_to_ids(doc_tokens) +
        [tokenizer.sep_token_id] +
        tokenizer.convert_tokens_to_ids(summ_tokens) +
        [tokenizer.sep_token_id]
    )
    
    # Truncate or pad to max_length
    if len(combined_tokens) > max_length:
        combined_tokens = combined_tokens[:max_length]
    
    attention_mask = [1] * len(combined_tokens)
    
    # Padding
    pad_length = max_length - len(combined_tokens)
    if pad_length > 0:
        combined_tokens += [tokenizer.pad_token_id] * pad_length
        attention_mask += [0] * pad_length
    
    return {
        "input_ids": torch.tensor([combined_tokens], dtype=torch.long),
        "attention_mask": torch.tensor([attention_mask], dtype=torch.long)
    }



def predict(text, summary, model, tokenizer):
    inputs = prepare_input_consistent(text, summary, tokenizer, max_length=512)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        prediction = torch.argmax(probabilities, dim=1).item()

    print(f"Prediction: {prediction}")
    print(f"Probabilities: {probabilities}")

    return prediction, probabilities



In [5]:
# Example text and summary
text = "Despite the hype surrounding its first watch, the iPhone is still the engine behind Apple's phenomenal success, its latest figures have revealed. The results far surpassed most analysts' expectations for the first three months of the year, when sales traditionally fall from their holiday-season peak. Apple sold more than 61 million iPhones in the quarter, accounting for more than two-thirds of its $58 billion in revenue for the quarter and the lion's share of its $13.6 billion in profit - and up 40% from a year ago . Sales of iPhones in China were also revealed to have outstripped those in the US. Apple sold more than 61 million iPhones in the quarter, accounting for more than two-thirds of its $58 billion in revenue for the quarter and the lion's share of its $13.6 billion in profit. $58 billion in revenue, $13.6 billion in profit . $200 billion in cash, up from around $150 billion a year ago. More than 61 million iPhones sole . iPad revenue fell 29% to $5.4 billion . Revenue from Mac computers rose 2%from a year earlier, to $5.6 billion . 'We are thrilled by the continued strength of iPhone, Mac and the App Store, which drove our best March quarter results ever,' said Tim Cook, Apple's CEO. 'We're seeing a higher rate of people switching to iPhone than we've experienced in previous cycles, and we're off to an exciting start to the June quarter with the launch of Apple Watch.' As expected, the numbers were down from the previous quarter, when holiday shoppers snapped up a record 74 million of Apple's new iPhone 6, 6 Plus and older models. But it was a 40 percent increase over the number of iPhones sold in the first three months of 2014. 'We're seeing great results all over the world,' Apple chief financial officer Luca Maestri told The Associated Press, adding that iPhone sales grew 72 percent in China, where the company has big hopes for expansion. Other products played a much smaller role. Revenue from Mac computers rose 2 percent from a year earlier, to $5.6 billion, while iPad revenue fell 29 percent, to $5.4 billion — continuing a steady decline in tablet sales. Apple didn't report any results for the new Apple Watch, which it began selling this month, after the quarter ended. Maestri said customer response had been 'positive.' Analysts estimate about 2 million have sold to date, suggesting early demand is healthy but not of blockbuster proportions. Apple shares have gained more than 50 percent over the last year, making it the world's most valuable company. 'It's been really great to see the reaction of customers,' said Cook. 'The response has been overwhelming. We can't wait to see more of the inspiring apps developers dream up.' The iPhone is another story. Since it began offering models with bigger screens last fall, Apple has vied with South Korea's Samsung for the No. 1 position in the global smartphone market. By some estimates, Apple outsold Samsung in the quarter that ended in December, and analysts will be watching closely when Samsung reports its latest results this week. Apple also announced an expansion of its effort to return more of its sizable cash war chest to investors. The company said it will raise its quarterly dividend by 11 percent, to 52 cents a share, and has increased a $90 billion stock buyback program to $140 billion. Apple didn't report any results for the new Apple Watch, which it began selling this month, after the quarter ended. In total, the company said the program will return $200 billion to investors by the end of March 2017. As iPhone sales have surged, so has Apple's stock. Apple shares have gained more than 50 percent over the last year, making it the world's most valuable company. The stock closed Monday at $132.65, up 1.8 percent for the day, and was rising in late trading. The iPhone isn't just Apple's 'dominant product,' said Frank Gillett, a tech industry analyst at Forrester Research. 'It's more than anything else what's driving the success of their company.' Market researchers, however, expect growth in the world smartphone market will slow this year, particularly at the higher price range where Apple competes, as most consumers in developed countries have already bought one. That could make it difficult for Apple to maintain its recent pace. 'They're extremely dependent on the iPhone,' said investment Colin Gillis at BGC Partners. 'At some point, the market dynamics change,' he said, adding that 'the question is what could replace the iPhone' if sales begin to slow. Customers looking at Apple iPhones in an Apple store in Shanghai, China, on January 14, 2014. Apple CEO Tim Cook has said he's optimistic about new markets such as China, where Apple has made a strong showing against Samsung and China's Xiaomi. And even if Apple is increasingly selling new iPhones to people who are simply upgrading older models, 'that's still a pretty healthy market,' said Gartner analyst Van Baker, noting that more than 700 million iPhones have been sold since the first model was introduced in 2007. Maestri also stressed the potential for new products like Apple Watch and Apple Pay, the company's mobile payment service. While these currently provide minimal revenue, analysts say they have big potential. And they are designed to work closely with the iPhone, which means each may bolster the other's popularity in the future, Gillett said.;'Apple sold more than 61 million iPhones in the quarter.'"
# summary = "Apple didn't report any results for the new Apple Watch . Believed around 2 million watches have been sold, according to estimates ."
summary_fake = ""

prediction, probabilities = predict(text, summary_fake, model, tokenizer)




Prediction: 0
Probabilities: tensor([[0.8960, 0.1040]])
